**TODO**
* Finalize method to store cross-validation results and visualize accuracy and stability.
* Visualize and report performance.
* Perform due diligence on methods and check in with collaborator.

### Import modules

In [414]:
import parameters as p

import numpy as np
import pandas as pd

import sklearn.datasets as datasets
import sklearn.linear_model as linear_model

import rpy2.robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

### Define functions

In [415]:
def make_synth_data(p):
    '''Create a synthetic dataset of variable size using sample generators from scikit-learn.
    The number of informative features and the target variable type (discrete or continuous) are
    also declareable parameters.
    '''
    if p.DISCRETE_Y:
        X, y = datasets.make_classification(n_samples=p.N_SAMPLES, n_features=p.N_FEATURES,
                                            n_informative=p.N_INFORMATIVE, n_redundant=0,
                                            n_repeated=0, n_classes=p.N_CLASSES,
                                            n_clusters_per_class=1, weights=p.WEIGHTS,
                                            flip_y=0.01, class_sep=1.0, hypercube=False,
                                            shift=0.0, scale=1.0, shuffle=True, random_state=None)
    else:
        X, y = datasets.make_regression(n_samples=p.N_SAMPLES, n_features=p.N_FEATURES,
                                        n_informative=p.N_INFORMATIVE, n_targets=1, bias=0.0,
                                        effective_rank=None, tail_strength=0.5, noise=0.0,
                                        shuffle=True, coef=False, random_state=None)
    
    # Transform X from NumPy array to pandas DataFrame
    X = pd.DataFrame(X, columns=['f{}'.format(i) for i in range(p.N_FEATURES)])
    
    return X, y

def assign_folds(X, p):
    '''Assign each unit in a dataset to a cross-validation fold using blockTools. The number of
    folds as well as blockTools parameters are declarable.
    '''
    # Activate pandas conversion support
    pandas2ri.activate()
    # Import blockTools
    blockTools = importr('blockTools')

    # Name blocking features
    block_vars = robjects.StrVector(X.columns)
    # Assign units to blocks and then folds
    blocks = blockTools.block(X.reset_index(), id_vars='index', block_vars=block_vars, n_tr=p.K)
    folds = blockTools.assignment(blocks)
    # Transform folds to DataFrame
    folds = pandas2ri.ri2py_dataframe(folds.rx2('assg').rx2('1')).astype(int)

    # Reformat DataFrame for cross-validation
    max_dist = folds.pop('Max Distance')
    folds.columns = np.arange(p.K)
    folds = folds.stack()
    folds.index = folds.index.droplevel(0)
    folds = folds.reset_index()
    folds.columns = ['k', 'unit']

    return folds, max_dist

def fit_model(X, y, folds, p):
    pass

### Perform analysis

In [416]:
# Create synthetic data
X, y = make_synth_data(p)

# Use blockTools for cross-validation assignment
folds, max_dist = assign_folds(X, p)

### Sandbox

In [417]:
model = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0,
                                        fit_intercept=True, intercept_scaling=1,
                                        class_weight=None, random_state=None, solver='liblinear',
                                        max_iter=100, multi_class='ovr', verbose=0,
                                        warm_start=False, n_jobs=1)

x = 0
for k in range(p.K):
    is_test = np.in1d(np.arange(p.N_SAMPLES), folds.ix[folds['k'] == k, 'unit'])
    model.fit(X.ix[~is_test, :], y[~is_test])
    x += model.score(X.ix[is_test, :], y[is_test])
x * 10

71.0

In [418]:
folds['unit_shuffled'] = folds['unit'].sample(frac=1).reset_index(drop=True)

In [419]:
model = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0,
                                        fit_intercept=True, intercept_scaling=1,
                                        class_weight=None, random_state=None, solver='liblinear',
                                        max_iter=100, multi_class='ovr', verbose=0,
                                        warm_start=False, n_jobs=1)

x = 0
for k in range(p.K):
    is_test = np.in1d(np.arange(p.N_SAMPLES), folds.ix[folds['k'] == k, 'unit_shuffled'])
    model.fit(X.ix[~is_test, :], y[~is_test])
    x += model.score(X.ix[is_test, :], y[is_test])
x * 10

66.0